## Лекция 2  BM5    

## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [1]:
import os
import csv

In [2]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymystem3 import Mystem

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Ya.Klop/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
mystem = Mystem()
stop_words = stopwords.words('russian')
tokenizer = RegexpTokenizer(r'[а-яА-ЯёЁ]+')

In [5]:
def preprocess_text(text):
    tokenized = tokenizer.tokenize(text)
    lemmatized = [mystem.lemmatize(token)[0] for token in tokenized]
    without_stopwords = [word for word in lemmatized if word not in stop_words]

    return " ".join(without_stopwords)

In [6]:
from tqdm import tqdm

queries = []
documents = []
original_documents = []
is_duplicates = []

with open(os.path.abspath('quora_question_pairs_rus.csv'), 'r', encoding='utf-8') as file:
    table = csv.reader(file)

    for row in tqdm(list(table)):
        if row[0] == '':
            continue
        # TODO: убрать
        if row[0] == '10000':
            break
        queries.append(preprocess_text(row[1]))
        documents.append(preprocess_text(row[2]))
        original_documents.append(row[2])
        is_duplicates.append(row[3])

print(queries[1], documents[1], is_duplicates[1], sep='\n')

  2%|▏         | 9987/404289 [00:38<32:20, 203.14it/s] 

мочь увеличивать скорость интернет соединение использовать
повышать скорость интернет путем взлом
0


### __Задача 1__:    
Напишите два поисковика на *BM25*. Один через подсчет метрики по формуле для каждой пары слово-документ, второй через умножение матрицы на вектор. 

Сравните время работы поиска на 100к запросах. В качестве корпуса возьмем 
[Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian).

In [9]:
from math import log
from statistics import mean

import pandas as pd

In [10]:
default_k = 2.0
default_b = 0.75

def bm25(inverted_index, query, num=10, k=default_k, b=default_b) -> float:
    normalized_query = preprocess_text(query).split()

    N = inverted_index.shape[1]
    avgdl = mean([len(document) for document in documents])

    result = []

    for word in normalized_query:
        nq = inverted_index.sum(axis=1)[word] if word in inverted_index.transpose() else 0

        for index, document in enumerate(documents):
            ld = len(document.split())
            tf = inverted_index[document][word] if word in inverted_index[document] else 0
            
            idf = log((N - nq + 0.5) / (nq + 0.5))
            score = idf * (tf * (k + 1)) / (tf + k * (1 - b + b * (ld / avgdl)))
            
            if score != 0:
                result.append((score, original_documents[index]))

    if (len(result) != 0):
        return sorted(result, key=lambda score_doc: score_doc[0], reverse=True)[:10]
    else:
        return "not found"

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
cv.fit(queries)

inverted_index = pd.DataFrame(
        cv.transform(queries).A,
        columns=cv.get_feature_names(),
        index=documents
    ).transpose()

inverted_index.head()

,происходить правительство индия украсть кохинор кох ноор алмаз назад,повышать скорость интернет путем взлом,находить остаток математика математика разделять,рыба выживать соленый вода,тройной луна козерог восхождение козерог это говорить обо,делать ребенок активный далекий телефонный видеоигр,должный делать великий геолог,использовать вместо,мочь взламывать бесплатный интернет,некоторые технический специалист мочь рассказывать долговечность надежность ноутбук компонент,...,каков различный тип раскладка,мировой война либо иметь место,думать мусульманин свинья,это летать первый класс,каков стоимость жизнь европа сравнение индийский город ченнай бангалор,каков различие прослушивание развлекательный компания также отличаться свой система прослушивание,каков шаг решение это уравнение,насколько хороший изучение,требоваться рекомендовать учебник курс органический химия бакалавриат,безопасно женщина путешествовать одиночка япония
аамми,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
абап,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
абдоминальный,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
аберденшир,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
абзац,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
bm25(inverted_index, 'солёная вода')

[(12.959955124491186, 'почему в Японии импортируется соленая вода'),
 (12.70745534612108, 'почему соленые таффи конфеты импортированы в Китае'),
 (12.70745534612108, 'почему тафтинка из соленой воды импортируется в италию'),
 (12.70745534612108,
  'почему тафтинги из соленой воды импортируются в португальском'),
 (12.114699217416875,
  'как распределяется водопроводная вода и система удаления сточных вод в Мумбаи'),
 (12.005729745064446,
  'почему соленая вода таффи конфеты, импортированные или неизвестные за пределами США'),
 (10.471608489454544, 'почему вода синяя'),
 (10.471608489454544, 'как производится электричество от воды'),
 (10.471608489454544, 'как мне получить работу в студии'),
 (10.471608489454544, 'вода практически несжимаема')]

### __Задача 2__:    



Выведите 10 первых результатов и их близость по метрике BM25 по запросу **рождественские каникулы** на нашем корпусе  Quora question pairs. 

In [13]:
bm25(inverted_index, 'рождественские каникулы')

[(16.181646119885823, 'каков ваш рождественский список'),
 (15.242658967659366, 'Какое лучшее место для отдыха на 10 дней'),
 (14.64871123032008,
  'которые являются лучшими местами в гоа, чтобы посетить их')]

### __Задача 3__:    

Посчитайте точность поиска при 
1. BM25, b=0.75 
2. BM15, b=0 
3. BM11, b=1

In [16]:
bm25(inverted_index, 'плаваем')

[(14.05425626750734, 'когда самое время научить своих детей плавать'),
 (14.05425626750734,
  'это грамматически некорректно, чтобы сказать, как она побежала, а не как она бежала'),
 (13.278158208683081,
  'когда мертвое тело находится в реке, сколько времени требуется, чтобы оно плавало')]

In [17]:
bm25(inverted_index, 'плаваем', b=0)

[(7.957327372225605,
  'когда мертвое тело находится в реке, сколько времени требуется, чтобы оно плавало'),
 (7.957327372225605, 'когда самое время научить своих детей плавать'),
 (7.957327372225605,
  'это грамматически некорректно, чтобы сказать, как она побежала, а не как она бежала')]

In [18]:
bm25(inverted_index, 'плаваем', b=1.01)

[(19.137504125565663, 'когда самое время научить своих детей плавать'),
 (19.137504125565663,
  'это грамматически некорректно, чтобы сказать, как она побежала, а не как она бежала'),
 (17.28489791112948,
  'когда мертвое тело находится в реке, сколько времени требуется, чтобы оно плавало')]